In [1]:
import pandas as pd
import numpy as np
from herramientas import obtener_resumen_valores_unicos
import plotly.express as px
import gc

In [2]:
#train = pd.read_csv('bbva-data-challenge-2023/archive/universe_train.csv')
#balances = pd.read_csv('bbva-data-challenge-2023/archive/balances.csv')
customers = pd.read_csv('bbva-data-challenge-2023/archive/customers.csv')
liabiities = pd.read_csv('bbva-data-challenge-2023/archive/liabilities.csv')
#movements = pd.read_csv('bbva-data-challenge-2023/archive/movements.csv')
#test = pd.read_csv('bbva-data-challenge-2023/archive/universe_test.csv')
#digital = pd.read_csv('bbva-data-challenge-2023/digital.csv')

In [3]:
df = pd.merge(customers, liabiities, on='ID', how='left')

In [4]:
# Reemplazar 'yes' con 1 y 'no' con 0 en ambas columnas
df['product_1_x'] = df['product_1_x'].replace({'Yes': 1, 'No': 0})
df['product_2_x'] = df['product_2_x'].replace({'Yes': 1, 'No': 0})
df['product_3'] = df['product_3'].replace({'Yes': 1, 'No': 0})
df['product_4'] = df['product_4'].replace({'Yes': 1, 'No': 0})
df['ofert_1'] = df['ofert_1'].replace({'Yes': 1, 'No': 0})
df['ofert_2'] = df['ofert_2'].replace({'Yes': 1, 'No': 0})
df['ofert_3'] = df['ofert_3'].replace({'Yes': 1, 'No': 0})

In [5]:
# Primero, crea variables dummy para columna1
df = pd.get_dummies(df, columns=['type_job'], prefix=['type_job'], drop_first=True)

# Luego, crea variables dummy para columna2
df = pd.get_dummies(df, columns=['bureau_risk'], prefix=['bureau_risk'], drop_first=True)

In [6]:

# Crear un DataFrame auxiliar con todos los pares posibles de ID y month
id_values = df['ID'].unique()
months = np.arange(1, 13)
pairs = [(id_val, month) for id_val in id_values for month in months]
aux_df = pd.DataFrame(pairs, columns=['ID', 'month'])

# Fusionar el DataFrame auxiliar con el DataFrame original para rellenar los valores faltantes
df = pd.merge(df, aux_df, on=['ID', 'month'], how='left')

# Ahora df contendrá todas las combinaciones de ID y month posibles, con valores faltantes rellenados con NaN
# Puedes reemplazar los NaN con 1 o cualquier otro valor predeterminado si lo deseas
df['month'].fillna(1, inplace=True)  # Reemplazar NaN con 1

# Verificar el resultado
# df

In [7]:
obtener_resumen_valores_unicos(df)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos
0,ID,object,809579,8895559,0
1,age,float64,15395,8895559,0
2,income,float64,14072,8895559,0
3,time_from_specialized,float64,20324,8895559,0
4,payroll,float64,22474,8895559,0
5,product_1_x,int64,2,8895559,0
6,product_2_x,int64,2,8895559,0
7,product_3,int64,2,8895559,0
8,product_4,int64,2,8895559,0
9,ofert_1,int64,2,8895559,0


In [8]:
df1 = df.copy()

In [9]:
## product_1_y	como target
# Eliminar la columna
# df = df.drop(columns=['product_2_y' ,'period'])
df = df.drop(columns=['period'])

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
# y quieres dividirlo en train y test basado en la columna "product_1_y"

# Primero, crea una columna llamada "set" que indique "train" si product_1_y no es nulo y "test" si lo es
df['set'] = df['product_1_y'].apply(lambda x: 'train' if pd.notna(x) else 'test')

# Ahora, puedes dividir el DataFrame en train y test utilizando la columna "set"
train_df = df[df['set'] == 'train'].drop(columns=['set'])  # Elimina la columna "set" después de dividir
test_df = df[df['set'] == 'test'].drop(columns=['set'])

# train_df contendrá las filas con valores no nulos en "product_1_y"
# test_df contendrá las filas con valores nulos en "product_1_y"

# A partir de este punto, puedes utilizar train_df y test_df para entrenar y probar tus modelos respectivamente.


In [12]:
test_df_1 = test_df.copy()

In [13]:
# Definir las variables predictoras (X)
x  = train_df.drop(columns=['product_1_y'])  # Elimina la columna 'product_1_y' del DataFrame

# Definir la variable objetivo (y)
y = train_df['product_1_y']

In [14]:
X = x.drop(columns=['ID'])

In [15]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un objeto XGBoost Regressor
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio (MSE) en el conjunto de prueba
mse = mean_squared_error(y_test, y_pred)
print(f"Error Cuadrático Medio (MSE): {mse}")

# Puedes ajustar los hiperparámetros y optimizar el modelo según sea necesario

/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

Error Cuadrático Medio (MSE): 0.9304123074497282


In [16]:
x1  = test_df.drop(columns=['product_1_y', 'ID']) 

In [17]:
x1

,age,income,time_from_specialized,payroll,product_1_x,product_2_x,product_3,product_4,ofert_1,ofert_2,...,type_job_type_6,type_job_type_7,bureau_risk_category_2,bureau_risk_category_3,bureau_risk_category_4,bureau_risk_category_5,bureau_risk_category_6,bureau_risk_category_7,bureau_risk_category_8,bureau_risk_category_9
96,4.8002,4.4046,4.6219,0.0,1,0,0,0,0,0,...,False,True,False,False,True,False,False,False,False,False
241,4.7539,4.2882,0.0000,0.0,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,True,False
571,4.6234,4.8570,4.9611,0.0,1,0,0,0,0,0,...,False,True,False,False,False,False,False,True,False,False
1475,4.4232,4.6935,4.4036,0.0,1,0,0,0,0,0,...,False,False,True,False,False,False,False,False,False,False
2097,4.8635,4.4597,0.0000,0.0,1,0,0,0,0,0,...,False,True,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8892253,4.5028,0.0000,0.0000,0.0,0,0,0,1,0,0,...,False,True,False,False,False,False,False,False,False,True
8892290,4.4714,4.3066,0.0000,0.0,1,0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,False
8893349,4.2654,4.5727,4.4603,0.0,1,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,False
8894244,4.0956,4.6955,4.2069,0.0,1,0,0,0,0,0,...,False,False,False,True,False,False,False,False,False,False


In [18]:
y_pred_1 = model.predict(x1)

/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

In [19]:
test_df = test_df.drop(columns=['product_1_y']) 

In [20]:
test_df = test_df.reset_index(drop=True)
test_df

,ID,age,income,time_from_specialized,payroll,product_1_x,product_2_x,product_3,product_4,ofert_1,...,type_job_type_6,type_job_type_7,bureau_risk_category_2,bureau_risk_category_3,bureau_risk_category_4,bureau_risk_category_5,bureau_risk_category_6,bureau_risk_category_7,bureau_risk_category_8,bureau_risk_category_9
0,d2e83db2dabfbe92577753a7ab9fd557df93e87cb4e3f8...,4.8002,4.4046,4.6219,0.0,1,0,0,0,0,...,False,True,False,False,True,False,False,False,False,False
1,bb577d0cafa0b878b8356535035629e56d197294f1fbba...,4.7539,4.2882,0.0000,0.0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,True,False
2,f005cae93b4a6b35f8b6a891556ccebb2025a5aba177ed...,4.6234,4.8570,4.9611,0.0,1,0,0,0,0,...,False,True,False,False,False,False,False,True,False,False
3,a8deca2d73e048bb094175e283691359f0d578ac7564c0...,4.4232,4.6935,4.4036,0.0,1,0,0,0,0,...,False,False,True,False,False,False,False,False,False,False
4,cf366cd55414cdb02d4efedbac276011b0ff9748a69f71...,4.8635,4.4597,0.0000,0.0,1,0,0,0,0,...,False,True,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18994,de6e8ac337c392358f70094ceceeb3770725500b0948ec...,4.5028,0.0000,0.0000,0.0,0,0,0,1,0,...,False,True,False,False,False,False,False,False,False,True
18995,e5d438faa90af28beed78f524e8e128343b4f180406d37...,4.4714,4.3066,0.0000,0.0,1,0,0,0,0,...,False,False,False,False,False,False,False,True,False,False
18996,4d9de82ed5dcffb8ec4c7eca9ebf1abf906d7af590adfa...,4.2654,4.5727,4.4603,0.0,1,0,0,1,0,...,False,False,False,False,False,False,False,True,False,False
18997,1fee90166e0ec5da225dd69029ad72bfab878cf1c0586c...,4.0956,4.6955,4.2069,0.0,1,0,0,0,0,...,False,False,False,True,False,False,False,False,False,False


In [21]:
y_pred_1 = pd.DataFrame(y_pred_1, columns=['product_1_y'])  


In [24]:

X_test = pd.concat([test_df, y_pred_1], axis=1)

In [25]:
X_test

,ID,age,income,time_from_specialized,payroll,product_1_x,product_2_x,product_3,product_4,ofert_1,...,type_job_type_7,bureau_risk_category_2,bureau_risk_category_3,bureau_risk_category_4,bureau_risk_category_5,bureau_risk_category_6,bureau_risk_category_7,bureau_risk_category_8,bureau_risk_category_9,product_1_y
0,d2e83db2dabfbe92577753a7ab9fd557df93e87cb4e3f8...,4.8002,4.4046,4.6219,0.0,1,0,0,0,0,...,True,False,False,True,False,False,False,False,False,2.408014
1,bb577d0cafa0b878b8356535035629e56d197294f1fbba...,4.7539,4.2882,0.0000,0.0,0,0,0,0,0,...,True,False,False,False,False,False,False,True,False,3.021418
2,f005cae93b4a6b35f8b6a891556ccebb2025a5aba177ed...,4.6234,4.8570,4.9611,0.0,1,0,0,0,0,...,True,False,False,False,False,False,True,False,False,1.895927
3,a8deca2d73e048bb094175e283691359f0d578ac7564c0...,4.4232,4.6935,4.4036,0.0,1,0,0,0,0,...,False,True,False,False,False,False,False,False,False,2.502657
4,cf366cd55414cdb02d4efedbac276011b0ff9748a69f71...,4.8635,4.4597,0.0000,0.0,1,0,0,0,0,...,True,False,False,False,False,False,False,True,False,2.890033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18994,de6e8ac337c392358f70094ceceeb3770725500b0948ec...,4.5028,0.0000,0.0000,0.0,0,0,0,1,0,...,True,False,False,False,False,False,False,False,True,0.434191
18995,e5d438faa90af28beed78f524e8e128343b4f180406d37...,4.4714,4.3066,0.0000,0.0,1,0,0,0,0,...,False,False,False,False,False,False,True,False,False,1.740378
18996,4d9de82ed5dcffb8ec4c7eca9ebf1abf906d7af590adfa...,4.2654,4.5727,4.4603,0.0,1,0,0,1,0,...,False,False,False,False,False,False,True,False,False,3.892130
18997,1fee90166e0ec5da225dd69029ad72bfab878cf1c0586c...,4.0956,4.6955,4.2069,0.0,1,0,0,0,0,...,False,False,True,False,False,False,False,False,False,3.003844


In [26]:
train_df

,ID,age,income,time_from_specialized,payroll,product_1_x,product_2_x,product_3,product_4,ofert_1,...,type_job_type_6,type_job_type_7,bureau_risk_category_2,bureau_risk_category_3,bureau_risk_category_4,bureau_risk_category_5,bureau_risk_category_6,bureau_risk_category_7,bureau_risk_category_8,bureau_risk_category_9
0,42af3bf64c2c14307361e970e76c9653df78b215838971...,4.2484,4.1259,0.0,4.8095,1,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False
1,42af3bf64c2c14307361e970e76c9653df78b215838971...,4.2484,4.1259,0.0,4.8095,1,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False
2,42af3bf64c2c14307361e970e76c9653df78b215838971...,4.2484,4.1259,0.0,4.8095,1,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False
3,42af3bf64c2c14307361e970e76c9653df78b215838971...,4.2484,4.1259,0.0,4.8095,1,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False
4,42af3bf64c2c14307361e970e76c9653df78b215838971...,4.2484,4.1259,0.0,4.8095,1,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8895554,ccb883a647ce35beb68e0dd8518643acca047564ba9464...,4.3142,4.1835,0.0,4.2870,1,0,0,1,1,...,False,False,False,False,True,False,False,False,False,False
8895555,ccb883a647ce35beb68e0dd8518643acca047564ba9464...,4.3142,4.1835,0.0,4.2870,1,0,0,1,1,...,False,False,False,False,True,False,False,False,False,False
8895556,ccb883a647ce35beb68e0dd8518643acca047564ba9464...,4.3142,4.1835,0.0,4.2870,1,0,0,1,1,...,False,False,False,False,True,False,False,False,False,False
8895557,ccb883a647ce35beb68e0dd8518643acca047564ba9464...,4.3142,4.1835,0.0,4.2870,1,0,0,1,1,...,False,False,False,False,True,False,False,False,False,False


In [27]:
X_train = pd.concat([train_df, X_test], axis=0)

In [28]:
X_train

,ID,age,income,time_from_specialized,payroll,product_1_x,product_2_x,product_3,product_4,ofert_1,...,type_job_type_6,type_job_type_7,bureau_risk_category_2,bureau_risk_category_3,bureau_risk_category_4,bureau_risk_category_5,bureau_risk_category_6,bureau_risk_category_7,bureau_risk_category_8,bureau_risk_category_9
0,42af3bf64c2c14307361e970e76c9653df78b215838971...,4.2484,4.1259,0.0000,4.8095,1,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False
1,42af3bf64c2c14307361e970e76c9653df78b215838971...,4.2484,4.1259,0.0000,4.8095,1,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False
2,42af3bf64c2c14307361e970e76c9653df78b215838971...,4.2484,4.1259,0.0000,4.8095,1,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False
3,42af3bf64c2c14307361e970e76c9653df78b215838971...,4.2484,4.1259,0.0000,4.8095,1,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False
4,42af3bf64c2c14307361e970e76c9653df78b215838971...,4.2484,4.1259,0.0000,4.8095,1,0,0,1,0,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18994,de6e8ac337c392358f70094ceceeb3770725500b0948ec...,4.5028,0.0000,0.0000,0.0000,0,0,0,1,0,...,False,True,False,False,False,False,False,False,False,True
18995,e5d438faa90af28beed78f524e8e128343b4f180406d37...,4.4714,4.3066,0.0000,0.0000,1,0,0,0,0,...,False,False,False,False,False,False,False,True,False,False
18996,4d9de82ed5dcffb8ec4c7eca9ebf1abf906d7af590adfa...,4.2654,4.5727,4.4603,0.0000,1,0,0,1,0,...,False,False,False,False,False,False,False,True,False,False
18997,1fee90166e0ec5da225dd69029ad72bfab878cf1c0586c...,4.0956,4.6955,4.2069,0.0000,1,0,0,0,0,...,False,False,False,True,False,False,False,False,False,False


In [29]:
obtener_resumen_valores_unicos(X_train)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos
0,ID,object,809579,8895559,0
1,age,float64,15395,8895559,0
2,income,float64,14072,8895559,0
3,time_from_specialized,float64,20324,8895559,0
4,payroll,float64,22474,8895559,0
5,product_1_x,int64,2,8895559,0
6,product_2_x,int64,2,8895559,0
7,product_3,int64,2,8895559,0
8,product_4,int64,2,8895559,0
9,ofert_1,int64,2,8895559,0


# produc_2

In [30]:
# Definir las variables predictoras (X)
x  = train_df.drop(columns=['product_2_y'])  # Elimina la columna 'product_2_y' del DataFrame

# Definir la variable objetivo (y)
y = train_df['product_2_y']

In [31]:
X = x.drop(columns=['ID'])

In [32]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un objeto XGBoost Regressor
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio (MSE) en el conjunto de prueba
mse = mean_squared_error(y_test, y_pred)
print(f"Error Cuadrático Medio (MSE): {mse}")

# Puedes ajustar los hiperparámetros y optimizar el modelo según sea necesario

/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

Error Cuadrático Medio (MSE): 1.4005137441976396


In [33]:
x1  = test_df_1.drop(columns=['product_2_y', 'ID']) 

In [34]:
x1

,age,income,time_from_specialized,payroll,product_1_x,product_2_x,product_3,product_4,ofert_1,ofert_2,...,type_job_type_6,type_job_type_7,bureau_risk_category_2,bureau_risk_category_3,bureau_risk_category_4,bureau_risk_category_5,bureau_risk_category_6,bureau_risk_category_7,bureau_risk_category_8,bureau_risk_category_9
96,4.8002,4.4046,4.6219,0.0,1,0,0,0,0,0,...,False,True,False,False,True,False,False,False,False,False
241,4.7539,4.2882,0.0000,0.0,0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,True,False
571,4.6234,4.8570,4.9611,0.0,1,0,0,0,0,0,...,False,True,False,False,False,False,False,True,False,False
1475,4.4232,4.6935,4.4036,0.0,1,0,0,0,0,0,...,False,False,True,False,False,False,False,False,False,False
2097,4.8635,4.4597,0.0000,0.0,1,0,0,0,0,0,...,False,True,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8892253,4.5028,0.0000,0.0000,0.0,0,0,0,1,0,0,...,False,True,False,False,False,False,False,False,False,True
8892290,4.4714,4.3066,0.0000,0.0,1,0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,False
8893349,4.2654,4.5727,4.4603,0.0,1,0,0,1,0,0,...,False,False,False,False,False,False,False,True,False,False
8894244,4.0956,4.6955,4.2069,0.0,1,0,0,0,0,0,...,False,False,False,True,False,False,False,False,False,False


In [36]:
y_pred_1 = model.predict(x1)

/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/bbchanchis/kaggle/bbva/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype)

In [37]:
test_df_1 = test_df_1.drop(columns=['product_2_y']) 

In [38]:
test_df_1 = test_df_1.reset_index(drop=True)

In [39]:
y_pred_1 = pd.DataFrame(y_pred_1, columns=['product_2_y'])  


In [40]:
X_test = pd.concat([test_df_1, y_pred_1], axis=1)

In [41]:
X_train_1 = pd.concat([train_df, X_test], axis=0)

In [42]:
obtener_resumen_valores_unicos(X_train_1)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos
0,ID,object,809579,8895559,0
1,age,float64,15395,8895559,0
2,income,float64,14072,8895559,0
3,time_from_specialized,float64,20324,8895559,0
4,payroll,float64,22474,8895559,0
5,product_1_x,int64,2,8895559,0
6,product_2_x,int64,2,8895559,0
7,product_3,int64,2,8895559,0
8,product_4,int64,2,8895559,0
9,ofert_1,int64,2,8895559,0
